In [1]:
import numpy as np
import torch
_ = torch.manual_seed(10)
import os
import math
from sbi import utils as utils
import sbi
from sbi import inference
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
import scipy.io as sio

ImportError: cannot import name 'ABCIndex' from 'pandas.core.dtypes.generic' (/Users/xiaoyuwang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/dtypes/generic.py)

In [ ]:
initial_data = sio.loadmat("BVCBM_3para_training_dataset.mat")
y_true = sio.loadmat("paper2_BVCBM_synthetic_datasets.mat")

In [ ]:
y = torch.from_numpy(y_true["sim1"]).to(torch.float32)
theta = torch.from_numpy(initial_data["theta_prior"]).to(torch.float32)
x = torch.from_numpy(initial_data["sims_prior"]).to(torch.float32)

lower = torch.ones(9)
lower[0],lower[1],lower[5],lower[6] = 0*lower[0], 0*lower[1],0*lower[5], 0*lower[6]
lower[4] = 2 * lower[4]

upper = torch.ones(9)
upper[2],upper[3],upper[7],upper[8] = 50*upper[2],24*32*upper[3],50*upper[7],24*32*upper[8]
upper[4] = 31 * upper[4]

priorDist = utils.BoxUniform(lower,upper)
def simulator(theta):
    return theta
_,prior = prepare_for_sbi(simulator,priorDist)

proposal = prior

In [ ]:
inference = SNPE(prior=prior, density_estimator='nsf')
density_estimator = inference.append_simulations(theta, x, proposal=proposal).train()
posterior = inference.build_posterior(density_estimator)

In [ ]:
from sbi import analysis as analysis

posterior_samples_syn1 = posterior.sample((10000,),x = y) 

_ = analysis.pairplot(
    posterior_samples_syn1, figsize=(10, 10)
)

In [ ]:
posterior = torch.load('BVCBM_synthetic2_30k_theta_sims.pth')
proposal = posterior.set_default_x(y[1, 0:25])

syn1_data_iter2 = sio.loadmat("BVCBM_SNPE_Synthetic_40kSimulation.mat")
theta1 = torch.from_numpy(syn1_data_iter2["theta_syn2"]).to(torch.float32)
x1 = torch.from_numpy(syn1_data_iter2["sims2"]).to(torch.float32)

In [ ]:
density_estimator = inference.append_simulations(theta1, x1, proposal=proposal).train()
posterior = inference.build_posterior(density_estimator)

In [ ]:
torch.save(posterior, 'BVCBM_synthetic2_40k_theta_sims.pth')

In [ ]:
from sbi import analysis as analysis

posterior_samples_syn1 = posterior.sample((10000,),x = y[1, 0:25]) 

_ = analysis.pairplot(
    posterior_samples_syn1, figsize=(10, 10)
)

In [ ]:
sio.savemat("BVCBM_SNPE_synthetic2_50kTheta.mat", {"theta_syn2": posterior_samples_syn1.numpy()})